In [2]:
import numpy as np
import functools 
import math
import torch
import sklearn
import sys
sys.path.insert(1,"./src")

from NNCPD import NNCPD, weights_H, Energy_Loss_Tensor2, Energy_Loss_Tensor, Recon_Loss, L21_Norm, outer_product, outer_product_np, PTF, random_NNCPD, Fro_Norm



import torch
import numpy as np
from matplotlib import pyplot as plt
import sys
sys.path.insert(1,"./src")

from NNCPD import NNCPD, weights_H, Energy_Loss_Tensor2, Energy_Loss_Tensor, Recon_Loss, L21_Norm, outer_product, outer_product_np, PTF, random_NNCPD, Fro_Norm
from lsqnonneg_module import LsqNonneg
from trainNNCPD import train
#
import torch.nn as nn
from torch.autograd import Variable
from writer import Writer


import tensorly as tl
from tensorly import unfold as tl_unfold
from tensorly.decomposition import parafac, non_negative_parafac

torch.set_default_tensor_type(torch.DoubleTensor)

c:\Users\samue\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Function to enerate data
def hnmfdatagen(cs, std=0):
    if len(cs) > 2:
        matrices = [np.random.rand(cs[i], cs[i+1]) for i in range(len(cs)-1)]
        return functools.reduce(np.matmul, matrices[1:], matrices[0]) + np.random.normal(0, std, [cs[0],cs[-1]])
        # we might want to scale our generated data

In [4]:
# Zero out proportion of generated entries
def zero_entries(X, p_miss, mecha="MCAR", opt=None, p_obs=None, q=None):
    """
    Generate missing values for specifics missing-data mechanism and proportion of missing values. 
    
    Parameters
    ----------
    X : torch.DoubleTensor or np.ndarray, shape (n, d)
        Data for which missing values will be simulated.
        If a numpy array is provided, it will be converted to a pytorch tensor.
    p_miss : float
        Proportion of missing values to generate for variables which will have missing values.
    mecha : str, 
            Indicates the missing-data mechanism to be used. "MCAR" by default, "MAR", "MNAR" or "MNARsmask"
    opt: str, 
         For mecha = "MNAR", it indicates how the missing-data mechanism is generated: using a logistic regression ("logistic"), quantile censorship ("quantile") or logistic regression for generating a self-masked MNAR mechanism ("selfmasked").
    p_obs : float
            If mecha = "MAR", or mecha = "MNAR" with opt = "logistic" or "quanti", proportion of variables with *no* missing values that will be used for the logistic masking model.
    q : float
        If mecha = "MNAR" and opt = "quanti", quantile level at which the cuts should occur.
    
    Returns
    ----------
    A dictionnary containing:
    'X_init': the initial data matrix.
    'X_incomp': the data with the generated missing values.
    'mask': a matrix indexing the generated missing values.s
    """
    
    to_torch = torch.is_tensor(X) ## output a pytorch tensor, or a numpy array
    if not to_torch:
        X = X.astype(np.float32)
        X = torch.from_numpy(X)
    
    if mecha == "MAR":
        mask = MAR_mask(X, p_miss, p_obs).double()
    elif mecha == "MNAR" and opt == "logistic":
        mask = MNAR_mask_logistic(X, p_miss, p_obs).double()
    elif mecha == "MNAR" and opt == "quantile":
        mask = MNAR_mask_quantiles(X, p_miss, q, 1-p_obs).double()
    elif mecha == "MNAR" and opt == "selfmasked":
        mask = MNAR_self_mask_logistic(X, p_miss).double()
    else:
        mask = (torch.rand(X.shape) < p_miss).double()
    
    X_nas = X.clone()
    X_nas[mask.bool()] = 0
    
    return {'X_init': X.double(), 'X_incomp': X_nas.double(), 'mask': mask}

In [5]:
X = hnmfdatagen([16, 8, 4, 16])
X = zero_entries(X, p_miss=0.33333333, mecha="MCAR")
display(X)

{'X_init': tensor([[5.9197, 3.7788, 1.9471, 6.1880, 3.7028, 7.8176, 7.6761, 4.9613, 7.8366,
          7.0022, 7.1768, 4.6472, 5.3645, 4.9568, 4.3226, 2.7710],
         [4.5516, 2.9403, 1.5013, 4.8703, 2.7980, 5.9765, 5.8359, 3.5637, 5.8184,
          5.3238, 5.2725, 3.6374, 4.0903, 3.6485, 3.2175, 2.0016],
         [4.3736, 2.7115, 1.4024, 4.7192, 2.9330, 5.7551, 5.4649, 3.2569, 5.6138,
          5.0511, 5.0265, 3.4907, 3.9077, 3.7277, 3.4112, 1.9519],
         [5.6417, 3.5281, 1.8336, 5.8667, 3.6077, 7.4445, 7.2431, 4.6846, 7.4591,
          6.6235, 6.8107, 4.4011, 5.0768, 4.8119, 4.2279, 2.6549],
         [3.8131, 2.5895, 1.2941, 4.1495, 2.3901, 5.0862, 5.0790, 3.2118, 5.1688,
          4.6532, 4.7514, 3.0944, 3.5890, 3.1524, 2.7771, 1.7924],
         [5.8610, 3.8782, 1.9643, 6.2531, 3.6061, 7.7639, 7.7067, 4.9037, 7.7900,
          7.0296, 7.1468, 4.6842, 5.3986, 4.7969, 4.1828, 2.7141],
         [4.5690, 2.8380, 1.4709, 4.8694, 2.9863, 6.0038, 5.7434, 3.4949, 5.8655,
          5.28

In [6]:
r=8
n1,n2 = X['X_incomp'].shape
net = NNCPD([n1,4],[n2,4],[1,1])

loss_func = Energy_Loss_Tensor()

history_unsupervised = train(net, X['X_incomp'], loss_func, r, epoch = 15000, lr1 = 0, lr2 = 0.1, random_init=True)


c:\Users\samue\AppData\Local\Programs\Python\Python39\lib\site-packages\torch\nn\_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


IndexError: You tried to access index 2 of a CP tensor.
You can only access index 0 and 1 of a CP tensor(corresponding respectively to the weights and factors)